<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Figure 4: Future projections

_Authors: Oriana Chegwidden and Jeremy Freeman_

The methods below conduct the analyses to recreate Figure 4 included the
manuscript <<< insert doi >>>. Future risks for fire. We show median changes in
risk with respect to the historic.


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import scipy as sp
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
import xarray as xr
from carbonplan_forests.utils import get_store


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from carbonplan_forests import load, setup, plot, fit, utils, prepare, collect
from showit import image
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import regionmask as rm
from scipy.stats import binom
import altair as alt

from palettable.colorbrewer.sequential import YlOrRd_9
import altair as alt

alt.themes.enable("carbonplan_light")

In [ ]:
coarsen = 8
store = "az"
tlim = ("1984", "2018")

In [ ]:
mask = load.mask(store=store, year=2001)

In [ ]:
nlcd = load.nlcd(store=store, year=2001, coarsen=coarsen, mask=mask)
nftd = load.nftd(store=store, area_threshold=1500, coarsen=coarsen, mask=mask)

In [ ]:
mask_coarsened = load.mask(store=store, year=2001, coarsen=coarsen)

In [ ]:
mtbs = load.mtbs(store=store, coarsen=coarsen, tlim=tlim, mask=mask)
# give it the same x/y coords as nftd (and nlcd and climate)
# this re-assignment will help us with some masking lower down
mtbs = mtbs.assign_coords({"x": nftd.x, "y": nftd.y})

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
for setup in ["_temp_precip"]:  # , '']:
    simulation = xr.open_zarr(
        get_store("carbonplan-scratch", "data/fire_future{}.zarr".format(setup))
    ).assign_coords({"x": mask_coarsened.x, "y": mask_coarsened.y})
    #     historical_ds = xr.open_zarr(get_store("carbonplan-scratch", "data/fire{}.zarr".format(setup))).assign_coords({"x": mask_coarsened.x, "y": mask_coarsened.y})
    coarsened_historical = (
        historical_ds.sel(time=slice("1989", "2018"))
        .coarsen(time=120)
        .sum()
        .load()
        / 10
    )
    historical_df = coarsened_historical.mean(dim=["x", "y"]).to_dataframe()
    decadal_averages = plot.fire.calc_decadal_averages(simulation)
    plot.fire.future_ts(
        decadal_averages.where(mask_coarsened),
        historical=historical_df,
        domain=(0, 0.05),
    )

# Original


In [ ]:
plot.fire.future_ts(
    decadal_averages.where(mask_coarsened),
    historical=historical_df,
    domain=(0, 0.05),
)

# Just temp


In [ ]:
plot.fire.future_ts(
    decadal_averages.where(mask_coarsened),
    historical=historical_df,
    domain=(0, 0.2),
)

# Temp + Precip


In [ ]:
plot.fire.future_ts(
    decadal_averages.where(mask_coarsened),
    historical=historical_df,
    domain=(0, 0.02),
)

# Temp + Precip + CWD


In [ ]:
plot.fire.future_ts(
    decadal_averages.where(mask_coarsened),
    historical=historical_df,
    domain=(0, 0.015),
)